# Function Calling with Llama 3.1: Getting started

In this example, we’ll demonstrate how to use function calling with the Llama 3.1 model. We’ll integrate a simple function called GetTime, which the model will be able to call when appropriate. The function attributes will be defined.

In [ ]:
# Install these required dependencies to run this notebook
!pip install python-dotenv==1.0.1
!pip install langchain-core==0.3.68
!pip install langchain-sambanova==0.2.0

In [ ]:
# Import libraries
import os
import sys
import json
from datetime import datetime
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field

from dotenv import load_dotenv

# Get absolute path for repo_dir and add it to the Python module search path
current_dir = os.getcwd()
repo_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(repo_dir)

from langchain_sambanova import ChatSambaNova

# Constants and configuration
load_dotenv(os.path.join(repo_dir, '.env'), override=True)
SAMBANOVA_API_KEY = os.getenv("SAMBANOVA_API_KEY")
SAMBANOVA_API_BASE = "https://api.sambanova.ai/v1"
MODEL = "Meta-Llama-3.3-70B-Instruct"

### Tools definition

In [2]:
# Get time tool definition

# Argument schema
class GetTimeSchema(BaseModel):
    """Returns current date, current time or both."""

    kind: str = Field(description='kind of information to retrieve "date", "time" or "both"')

# Tool definition using @tool decorator
@tool(args_schema=GetTimeSchema)
def get_time(kind: str = "both") -> str:
    """Returns current date, current time or both.
    Args:
        kind(str): date, time or both
    
    Returns:
        str: current date, current time or both    
    """
    if kind == "date":
        date = datetime.now().strftime("%m/%d/%Y")
        return f"Current date: {date}"
    elif kind == "time":
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current time: {time}"
    else:
        date = datetime.now().strftime("%m/%d/%Y")
        time = datetime.now().strftime("%H:%M:%S")
        return f"Current date: {date}, Current time: {time}"
    
# Define list of tools
tools = [get_time]

# Define tool invocation method
def invoke_tools(tool_calls, messages):
    available_functions={tool.name:tool for tool in tools}
    for tool_call in tool_calls:
        selected_tool = available_functions[tool_call["name"]]
        tool_output = selected_tool.invoke(tool_call["args"])
        print(f"Tool output: {tool_output}")
        messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
    return messages

### Define the function calling model

In [ ]:
# Define tool calling model
llm = ChatSambaNova(
    sambanova_api_base= SAMBANOVA_API_BASE,
    sambanova_api_key = SAMBANOVA_API_KEY,
    model=MODEL,
    max_tokens=1024,
    temperature=0.7,
    top_p=0.01
)

llm_with_tools = llm.bind_tools(tools=tools)

### Call the function calling model

In [4]:
# Define messages
messages = [
    HumanMessage(
       content="I need to schedule a meeting for two weeks from today. Can you tell me the exact date of the meeting?"
    )
]

# Iterative pipeline
response = llm_with_tools.invoke(messages)
while len(response.tool_calls)>0:
    print(f"Intermediate model response: {response.tool_calls}")
    messages.append(response)
    messages = invoke_tools(response.tool_calls, messages)
    response = llm_with_tools.invoke(messages)

print(f"final response: {response.content}")

Intermediate model response: [{'name': 'get_time', 'args': {'kind': 'date'}, 'id': 'call_877ad8e1d9564235b7', 'type': 'tool_call'}]
Tool output: Current date: 10/14/2025
final response: Two weeks from the current date (10/14/2025) would be 10/28/2025.
